## Examples of PySpark ML

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('ML_Example').getOrCreate()
spark

In [4]:
## Read the dataset
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols = ['age','Experience'],
                                    outputCol = 'Independent Features')

In [9]:
output = feature_assembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data = output.select('Independent Features','Salary')
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression

## train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [15]:
## Coefficients
regressor.coefficients

DenseVector([-323.2867, 1696.8066])

In [16]:
## Intercepts
regressor.intercept

22295.299605312008

In [18]:
## Prediction
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17203.085755292603|
+--------------------+------+------------------+



In [19]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2203.0857552926027, 4853586.845173177)